In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import pprint
import torch.nn as nn

import matplotlib.pyplot as plt
%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv1d, Conv2d,MaxPool1d, MaxPool2d, Module, Softmax, BatchNorm1d, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

# torchvision for pre-trained models
from torchvision import models


In [2]:
master_csv = pd.read_csv(r"C:\Users\sonam\University of Canberra\4th Semester\Capstone\Models\Session_1_labels.csv")
master_csv.head()
master_csv.shape


(360, 4)

In [3]:
cwd = os.getcwd() 
folder_path = os.path.join(cwd,"session_1_all_200")

eeg_data=[]
for filename in tqdm(master_csv['filename']):
    file_path = os.path.join(folder_path, filename)
    file = pd.read_csv(file_path)
    
     # Drop the first row (column headers)
    file = file.iloc[1:]
    
    # Drop rows below row 6002
    file = file.iloc[:6002]
    
    file = file.astype('float32')
    eeg_values = file.drop(columns=['time']).values  # Assuming 'time' column is not needed
    
#     # Calculate mean and standard deviation for Z-score normalization
#     mean = np.mean(eeg_values, axis=0)
#     std_dev = np.std(eeg_values, axis=0)
    
#     # Apply Z-score normalization
#     eeg_values_normalized = (eeg_values - mean) / std_dev
    
    # Transpose the EEG data
    eeg_values= eeg_values.T
    
    eeg_data.append(eeg_values)
# converting the list to numpy array
eeg_numpy = np.array(eeg_data)
eeg_numpy.shape



100%|████████████████████████████████████████████████████████████████████████████████| 360/360 [03:30<00:00,  1.71it/s]


(360, 62, 6002)

In [37]:
#eeg_data

In [4]:
eeg_numpy

array([[[ 1.8298626e+07,  1.3113022e+07,  1.2934208e+07, ...,
         -7.7784060e+06, -1.3977289e+07, -1.7225742e+07],
        [ 1.9699336e+07,  1.3023615e+07,  2.1100044e+07, ...,
         -1.1712313e+07, -1.8119812e+07, -1.1950731e+07],
        [ 1.6123056e+07,  1.2725592e+07,  1.7762184e+07, ...,
         -3.1501054e+07, -3.5375360e+07, -3.0547380e+07],
        ...,
        [ 2.0742416e+07,  2.4825334e+07,  3.6150216e+07, ...,
         -2.5898218e+07, -1.7911196e+07, -1.4126301e+07],
        [ 2.2798776e+07,  2.6345252e+07,  3.4958124e+07, ...,
         -1.4543533e+07, -4.6491625e+06, -9.2387200e+05],
        [ 3.3259392e+07,  3.9786104e+07,  5.0812956e+07, ...,
         -2.3096800e+07, -1.2308359e+07, -1.0937452e+07]],

       [[-3.8146972e+06, -2.1159650e+06, -1.7225742e+07, ...,
         -2.6166440e+07, -3.0547380e+07, -2.3365020e+07],
        [ 1.8477440e+06, -5.0663947e+05, -1.2397766e+07, ...,
         -1.8745660e+07, -2.5302172e+07, -2.2500754e+07],
        [ 8.5234640e+06, 

In [5]:
eeg_numpy.shape

(360, 62, 6002)

In [6]:
# defining the target
label_emotion = master_csv['emotion'].values
print(label_emotion[:48])  # Display the first 10 elements

[1 2 3 0 2 0 0 1 0 1 2 1 1 1 2 3 2 2 3 3 0 3 0 3 1 2 3 0 2 0 0 1 0 1 2 1 1
 1 2 3 2 2 3 3 0 3 0 3]


In [27]:
# create validation set
train_x, val_x, train_y, val_y = train_test_split(eeg_numpy, label_emotion, test_size = 0.25, random_state = 13, stratify=label_emotion)
(train_x.shape, train_y.shape), (val_x.shape, val_y.shape)

(((270, 62, 6002), (270,)), ((90, 62, 6002), (90,)))

In [28]:
# converting training images into torch format
# Reshape training data

train_x = torch.from_numpy(train_x)

In [29]:
# converting the target into torch format
train_y = train_y.astype(int)
train_y = torch.from_numpy(train_y)


In [30]:
# shape of training data
train_x.shape, train_y.shape

(torch.Size([270, 62, 6002]), torch.Size([270]))

In [31]:
# Reshape test data

val_x = torch.from_numpy(val_x)

# converting the target into torch format
val_y = val_y.astype(int)
val_y = torch.from_numpy(val_y)


In [34]:
# shape of validation data
val_x.shape, val_y.shape

(torch.Size([90, 62, 6002]), torch.Size([90]))

In [36]:
# Assuming val_x is your input data
val_x = val_x.unsqueeze(1)  # Add a dummy channel dimension

# Shape of validation data after modification
val_x.shape

torch.Size([90, 1, 62, 6002])

In [38]:

class Net(nn.Module):   
    def __init__(self, in_channels=1, num_classes=4):
        super(Net, self).__init__()
        
        # Convolutional layers
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),                                      
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Calculate the number of features after convolutional layers
        self.num_features = self._calculate_num_features((62, 6002))  
        
        # Fully connected layers
        self.linear_layers = nn.Sequential(
            nn.Linear(self.num_features, 2048),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(2048, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes)
        )
    
    def _calculate_num_features(self, input_dim):
        # Forward pass to get the shape after convolutional layers
        input_tensor = torch.zeros(1, *input_dim)
        conv_output = self.cnn_layers(input_tensor)
        num_features = conv_output.view(1, -1).size(1)
        return num_features
    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

# Instantiate the model
model = Net()

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

ValueError: expected 4D input (got 3D input)

In [22]:
# defining the model
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.0001)

scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available

#if torch.cuda.is_available():
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = criterion.to(device)

print(model)

# batch size of the model
batch_size = 200

# number of epochs to train the model
n_epochs = 20


RuntimeError: Given input size: (128x750x1). Calculated output size: (128x375x0). Output size is too small

In [56]:
for epoch in range(1, n_epochs+1):
    
    scheduler.step()

    # keep track of training and validation loss
    train_loss = 0.0
        
    permutation = torch.randperm(train_x.size()[0])

    training_loss = []
    for i in tqdm(range(0,train_x.size()[0], batch_size)):

        indices = permutation[i:i+batch_size]
        batch_x, batch_y = train_x[indices], train_y[indices]
        
        # Convert batch_y to Long data type
        batch_y = batch_y.long()  # Convert to Long data type
        
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        # in case you wanted a semi-full example
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)

        training_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        
    training_loss = np.average(training_loss)
    print('epoch: \t', epoch, '\t training loss: \t', training_loss)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.84s/it]


epoch: 	 1 	 training loss: 	 1.3872495889663696


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.86s/it]


epoch: 	 2 	 training loss: 	 1.355642557144165


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.73s/it]


...

epoch: 	 18 	 training loss: 	 1.1094833612442017


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.67s/it]


epoch: 	 19 	 training loss: 	 1.127306580543518


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.75s/it]

epoch: 	 20 	 training loss: 	 1.1192717552185059


In [57]:
# prediction for training set
prediction = []
target = []
permutation = torch.randperm(train_x.size()[0])
for i in tqdm(range(0,train_x.size()[0], batch_size)):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = train_x[indices], train_y[indices]

    #if torch.cuda.is_available():
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    with torch.no_grad():
        output = model(batch_x)

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction.append(predictions)
    target.append(batch_y)
    
# training accuracy
accuracy = []
for i in range(len(prediction)):
    accuracy.append(accuracy_score(target[i],prediction[i]))
    
print('training accuracy: \t', np.average(accuracy))


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.03s/it]

training accuracy: 	 0.5857142857142856


In [58]:
# prediction for validation set
prediction_val = []
target_val = []
permutation = torch.randperm(val_x.size()[0])
for i in tqdm(range(0,val_x.size()[0], batch_size)):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = val_x[indices], val_y[indices]

    #if torch.cuda.is_available():
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    with torch.no_grad():
        output = model(batch_x)

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction_val.append(predictions)
    target_val.append(batch_y)
    
# validation accuracy
accuracy_val = []
for i in range(len(prediction_val)):
    accuracy_val.append(accuracy_score(target_val[i],prediction_val[i]))
    
print('validation accuracy: \t', np.average(accuracy_val))

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.68it/s]

validation accuracy: 	 0.3888888888888889
